The goal for this notebook is to determine how many coadds a particular visit will contribute to, then to build those coadds (including other possible visits).

In [1]:
REPO = '/sdf/data/rubin/repo/main/'

from lsst.daf.butler import Butler
from pprint import pprint
import lsst.afw.image as afwImage
from lsst.skymap import Index2D
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

%matplotlib inline

butler = Butler(REPO)
registry = butler.registry

# define image display backend
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')

In [2]:
input_collection = 'HSC/runs/RC2/w_2024_22/DM-44526'

In [3]:
# visit of interest
visit_y = butler.get('calexp',
                        collections = input_collection,
                        instrument='HSC',
                        detector=0,
                        visit = 318,
                        band = 'y',
                        day_obs = 20140325,
                        physical_filter = 'HSC-Y',
                       )

In [15]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(visit_y.getMaskedImage())

Before going much further, the visit will need to be warped first to match coordinates with the coadds later. To do this, I run the command:
```
pipetask run -j 4 --register-dataset-types \
-b /sdf/data/rubin/repo/main \
-i HSC/runs/RC2/w_2024_22/DM-44526 \
-o u/$USER/makeWarp_y_band \
-p /sdf/group/rubin/user/mgorsuch/cell-coadd/pipeline-directwarp.yaml \
-d "instrument='HSC' AND detector=0 AND visit=318 AND day_obs=20140325 AND band='y' AND physical_filter='HSC-Y' AND skymap='hsc_rings_cells_v1'"
```
This produces 5 warps, as the visit will overlap several combinations of tracks and patches.

In [4]:
warp_collection = "u/mgorsuch/makeWarp_y_band/20240618T185910Z"

In [35]:
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', collections=warp_collection,):
    print(ref.dataId)

{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9570, patch: 110, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9570, patch: 110, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9571, patch: 119, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9571, patch: 119, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9571, patch: 120, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9571, patch: 120, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 3, visit: 318, band: 'y', day_obs: 20140325, physical_filter: 'HSC-Y'}
{

In [11]:
warp = butler.get('deepCoadd_directWarp',
                   collections = warp_collection,
                   instrument='HSC',
                   skymap = 'hsc_rings_cells_v1',
                   tract = 9571,
                   patch = 120,
                   visit = 318)

In [12]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(visit_y.image)
display2 = afwDisplay.Display(frame=2)
display2.mtv(warp.image)